<a href="https://colab.research.google.com/github/marquito3012/TFG/blob/main/FineTuningActionClassificatorYolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics roboflow

# Import YOLO library
from ultralytics import YOLO

# Import Roboflow
from roboflow import Roboflow

# Import OS
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
HOME = os.getcwd()
print(HOME)
!mkdir {HOME}/datasets
%cd {HOME}/datasets

# Download the dataset
rf = Roboflow(api_key="9KG8ovuhVxeymK6aMBor")
project = rf.workspace("uni-mallw").project("drowsy-detection-qnv9b")
version = project.version(3)
dataset = version.download("yolov11")

dataset_path = dataset.location  # Asegúrate de que esta variable tiene el path correcto

# Lista archivos dentro del dataset
print("Contenido del dataset:", os.listdir(dataset_path))

import shutil
import yaml

dataset_path = dataset.location  # Asegúrate de que esta variable contiene la ruta correcta
output_path = f"{HOME}/datasets/classification_dataset"

# Cargar data.yaml para obtener los nombres de las clases
with open(os.path.join(dataset_path, "data.yaml"), "r") as f:
    data = yaml.safe_load(f)
class_names = data["names"]

# Procesar train y valid
for split in ["train", "valid"]:
    split_path = os.path.join(dataset_path, split)
    images_path = os.path.join(split_path, "images")
    labels_path = os.path.join(split_path, "labels")

    # Crear carpeta de salida para clasificación
    split_output_path = os.path.join(output_path, split)
    os.makedirs(split_output_path, exist_ok=True)

    # Crear carpetas de clases
    for class_name in class_names:
        os.makedirs(os.path.join(split_output_path, class_name), exist_ok=True)

    # Mover imágenes a sus respectivas carpetas de clase
    for label_file in os.listdir(labels_path):
        img_name = label_file.replace(".txt", ".jpg")  # Ajustar si las imágenes son PNG
        img_path = os.path.join(images_path, img_name)

        with open(os.path.join(labels_path, label_file), "r") as f:
            lines = f.readlines()

        if lines:
            class_id = int(lines[0].split()[0])  # Tomamos la primera clase detectada
            class_name = class_names[class_id]

            # Copiar la imagen a su respectiva carpeta de clase
            shutil.copy(img_path, os.path.join(split_output_path, class_name, img_name))

print("✅ Conversión completada. Nuevo dataset en:", output_path)

%cd classification_dataset
yaml_content = """train: train
val: valid
nc: 2
names: ['Awake', 'Drowsy']
"""

with open("data.yaml", "w") as file:
    file.write(yaml_content)

print("Archivo data.yaml creado exitosamente.")
%cd {HOME}

/content
/content/datasets
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Drowsy-Detection-3 in yolov11:: 100%|██████████| 3605/3605 [00:01<00:00, 2144.93it/s]


Contenido del dataset: ['valid', 'train', 'README.dataset.txt', 'data.yaml', 'README.roboflow.txt']
✅ Conversión completada. Nuevo dataset en: /content/datasets/classification_dataset
/content/datasets/classification_dataset
Archivo data.yaml creado exitosamente.
/content


In [ ]:
!cat /content/datasets/classification_dataset/data.yaml

train: train
val: valid
nc: 2
names: ['Awake', 'Drowsy']


In [ ]:
# Train YOLOv8 on your dataset
model = YOLO("yolo11n-cls.pt")

# Updated data path
data_path = f"{HOME}/datasets/classification_dataset"

train_results = model.train(
    data=data_path,  # Dataset path
    epochs=10,  # Adjust based on need
    batch=8,   # Adjust based on GPU
    plots = True
)

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/datasets/classification_dataset, epochs=10, time=None, patience=100, batch=8, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

train: Scanning /content/datasets/classification_dataset/train... 1256 images, 0 corrupt: 100%|██████████| 1256/1256 [00:00<?, ?it/s]


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
model.val()

In [ ]:
# Test on an image
results = model("/content/Drowsy-Detection-3/valid/images/221023-091721_jpg.rf.a92ac80ed2062adad9cdbf2034051b69.jpg", save=True)  # Replace with an actual test image path